In [6]:
!pip install pyowm

     |████████████████████████████████| 3.1 MB 297 kB/s eta 0:00:01


In [2]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 454 kB/s eta 0:00:01


In [4]:
!pip install pyttsx3

In [8]:
!pip install youtube-dl

     |████████████████████████████████| 1.9 MB 413 kB/s eta 0:00:01


In [20]:
!pip install python-vlc

In [28]:
!pip install urllib3

In [10]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11686 sha256=107df07c3b8633ecbff247dca5a381d861fd8cb41fcf3bfb52dc779688763e6e
  Stored in directory: /home/karthikn/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [39]:
import speech_recognition as sr
import pyttsx3
import os
import sys
import re
import webbrowser
import smtplib
import requests
import subprocess
from pyowm import OWM
import youtube_dl
#import vlc
import urllib
#import urllib2
import urllib.request as urllib2
import json
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import wikipedia
import random
from time import strftime
import datetime

In [13]:
def sofiaResponse(audio):
    "speaks audio passed as argument"
    print(audio)
    engine = pyttsx3.init()
    engine.say(audio)
    engine.runAndWait()
    
    for line in audio.splitlines():
        os.system("say " + audio)

In [14]:
def myCommand():
    "listens for commands"
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print('Say something...')
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1)
        audio = r.listen(source)
    try:
        command = r.recognize_google(audio).lower()
        print('You said: ' + command + '\n')
    except sr.UnknownValueError:
        print('....')
        command = myCommand();
    return command

In [15]:
def reddit(command):    
    reg_ex = re.search('open reddit (.*)', command)
    url = 'https://www.reddit.com/'
    if reg_ex:
        subreddit = reg_ex.group(1)
        url = url + 'r/' + subreddit
        
    webbrowser.open(url)
    sofiaResponse('The Reddit content has been opened for you Sir.')

In [16]:
def shutDown():
    sofiaResponse('Bye bye Sir. Have a nice day')
    sys.exit()

In [17]:
def openWebsite(command):
    reg_ex = re.search('open (.+)', command)
    if reg_ex:
        domain = reg_ex.group(1)
        print(domain)
        url = 'https://www.' + domain + ".com"
        webbrowser.open(url)
        sofiaResponse('The website you have requested has been opened for you Sir.')
    else:
        pass

In [18]:
def greetings():
    day_time = int(strftime('%H'))
    if day_time < 12:
        sofiaResponse('Hello Sir. Good morning')
    elif 12 <= day_time < 18:
        sofiaResponse('Hello Sir. Good afternoon')
    else:
        sofiaResponse('Hello Sir. Good evening')

In [19]:
def helpMe():
    sofiaResponse("""
            You can use these commands and I'll help you out:1. Open reddit subreddit : Opens the subreddit in default browser.
            2. Open xyz.com : replace xyz with any website name
            3. Send email/email : Follow up questions such as recipient name, content will be asked in order.
            4. Current weather in {cityname} : Tells you the current condition and temperture
            5. Hello
            6. play me a video : Plays song in your VLC media player
            7. change wallpaper : Change desktop wallpaper
            8. news for today : reads top news of today
            9. time : Current system time
            10. top stories from google news (RSS feeds)
            11. tell me about xyz : tells you about xyz""")

In [20]:
def joke():
    res = requests.get(
                'https://icanhazdadjoke.com/',
                headers={"Accept":"application/json"})
    if res.status_code == requests.codes.ok:
        sofiaResponse(str(res.json()['joke']))
    else:
        sofiaResponse('oops!I ran out of jokes')

In [21]:
def news(): 
    try:
        news_url="https://news.google.com/news/rss"
        Client=urlopen(news_url)
        xml_page=Client.read()
        Client.close()
        soup_page=soup(xml_page,"xml")
        news_list=soup_page.findAll("item")
        for news in news_list[:15]:
            sofiaResponse(news.title.text.encode('utf-8'))
    except Exception as e:
        print(e)

In [22]:
def weather(command):
    reg_ex = re.search('weather in (.*)', command)
    if reg_ex:
        city = reg_ex.group(1)
        owm = OWM('ab0d5e80e8dafb2cb81fa9e82431c1fa')
        mgr = owm.weather_manager()
        obs = mgr.weather_at_place(city)
        w = obs.weather
        k = w.detailed_status 
        x = w.temperature(unit='celsius')
        sofiaResponse('Current weather in %s is %s.\n \
            The maximum temperature is %0.2f and the\n \
            minimum temperature is %0.2f degree celcius\n' % (city, k, x['temp_max'], x['temp_min']))

In [23]:
def time():
    now = datetime.datetime.now()
    sofiaResponse('Current time is %d hours %d minutes' % (now.hour, now.minute))

In [24]:
def email():
    sofiaResponse('Who is the recipient?')
    recipient = myCommand()
    if 'rajat' in recipient:
        sofiaResponse('What should I say to him?')
        content = myCommand()
        mail = smtplib.SMTP('smtp.gmail.com', 587)
        mail.ehlo()
        mail.starttls()
        mail.login('your_email_address', 'your_password')
        mail.sendmail('sender_email', 'receiver_email', content)
        mail.close()
        sofiaResponse('Email has been sent successfuly. You can check your inbox.')
    else:
        sofiaResponse('I don\'t know what you mean!')

In [25]:
def launch(command):
    reg_ex = re.search('launch (.*)', command)
    if reg_ex:
        appname = reg_ex.group(1)
        #appname = appname.title()
        appname1 = appname+".exe"
        subprocess.Popen(["open", "-n", "/Applications/" + appname1], stdout=subprocess.PIPE)
        sofiaResponse('I have launched the desired application')

In [26]:
def playSong():
    
    path="D:\\"
    
    mysong = "kanda vara sollunga status"
    if mysong:
        flag = 0
        url = "https://www.youtube.com/results?search_query=" + mysong.replace(' ', '+')
        response = urllib2.urlopen(url)
        html = response.read()
        soup1 = soup(html,"lxml")
        url_list = []
        for vid in soup1.findAll(attrs={'class':'yt-uix-tile-link'}):
            if ('https://www.youtube.com' + vid['href']).startswith("https://www.youtube.com/watch?v="):
                flag = 1
                final_url = 'https://www.youtube.com' + vid['href']
                url_list.append(final_url)
                url = url_list[0]
        ydl_opts = {}
        os.chdir(path)
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        vlc.play(path)
        if flag == 0:
            sofiaResponse('I have not found anything in Youtube ')

In [27]:
def changeWallpaper():
    folder = '/Users/karthikn/Documents/wallpaper/'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)
    api_key = 'fd66364c0ad9e0f8aabe54ec3cfbed0a947f3f4014ce3b841bf2ff6e20948795'
    url = 'https://api.unsplash.com/photos/random?client_id=' + api_key
    f = urllib2.urlopen(url)
    json_string = f.read()
    f.close()
    parsed_json = json.loads(json_string)
    photo = parsed_json['urls']['full']
    urllib2.urlretrieve(photo, "/Users/karthikn/Documents/wallpaper/a")
    subprocess.call(["killall Dock"], shell=True)
    sofiaResponse('wallpaper changed successfully')

In [28]:
def wiki(command):
    reg_ex = re.search('tell me about (.*)', command)
    try:
        if reg_ex:
            topic = reg_ex.group(1)
            ny = wikipedia.page(topic)
            sofiaResponse(ny.content[:500].encode('utf-8'))
        
    except Exception as e:
        print(e)
        sofiaResponse(e)
        sofiaResponse('Hi User, I am Sofia and I am your personal voice assistant,\
        Please give a command or say "help me" and I will tell you what all I can do for you.')

In [40]:

def assistant(command):
    
    if 'open reddit' in command:
        reddit(command)
        
    elif 'shutdown' in command:
        shutDown()
        
    elif 'open' in command:
        openWebsite(command)
        
        
    elif 'hello' in command:
        greetings()
        
    elif 'help me' in command:
        helpMe()
        
    elif 'joke' in command:
        joke()
       
    elif 'news' in command:
        news()
        
    elif 'weather' in command:
        weather(command)

    elif 'time' in command:
        time()

    elif 'email' in command:
        email()

    elif 'launch' in command:
        launch(command)

    elif 'play me a song' in command:
        playSong()

    elif 'change wallpaper' in command:
        changeWallpaper()
        
        
    elif 'tell me about' in command:
        wiki(command)

    else:
        print("I can't understand what you are  saying")

        
print("Welcome Boss")
print("Enter 1 for command 2 for speech")
op = input()
op=int(op)
while True:
    if(op==1):
        print("Enter command")
        command = input()
        assistant(command)
    else:
        assistant(myCommand())

Welcome Boss
Enter 1 for command 2 for speech


 2


AttributeError: Could not find PyAudio; check installation

In [36]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/karthikn/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
sentence = "What is Weather in in TamilNadu"
tokens = nltk.word_tokenize(sentence)
tokens

['What', 'is', 'Weather', 'in', 'in', 'TamilNadu']